In [4]:
import sys
from pathlib import Path
from os.path  import join
from src.config import DATA_DIR, CONFIG_DIR, MODELS_DIR
from omegaconf import OmegaConf
from src.external.hptr.src.data_modules.agent_centric import AgentCentricPreProcessing
from src.external.hptr.src.data_modules.ac_global import AgentCentricGlobal
from src.mimolm import InputProjections, EarlyFusionEncoder, MotionDecoder
import torch
import torch.nn.functional as F
import lightning as pl

# Add the project root to sys.path
project_root = Path().resolve().parent  # Adjust as needed to point to the root folder
sys.path.append(str(project_root))

print(Path.cwd())  # Check if the path is added

/home/harshavardhan-patil/Work/Projects/mimolm/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/home/harshavardhan-patil/Work/Projects/mimolm/notebooks


In [5]:
from src.external.hptr.src.data_modules.data_h5_av2 import DataH5av2
from src.mimolm import MimoLM
from src.modeling.modules.lm_utils import interpolate_trajectory, cluster_rollouts, non_maximum_suppression

torch.set_printoptions(sci_mode=False)
data_module = DataH5av2(DATA_DIR)
data_module.setup(stage="validate")
val_loader = data_module.val_dataloader()

model = MimoLM.load_from_checkpoint(checkpoint_path=str(MODELS_DIR / "v2Me59.ckpt" )
                                    , data_size=data_module.tensor_size_val
                                    , n_rollouts = 64
                                    , learning_rate = 8.e-8)
trainer = pl.Trainer(fast_dev_run=1)
output = trainer.test(model=model, dataloaders=val_loader)

ValueError: expected sequence of length 3 at dim 1 (got 0)

In [13]:
import torch

token = torch.randint(100, (2, 3, 5, 1)) / 100.
token

tensor([[[[0.2900],
          [0.6300],
          [0.4900],
          [0.5200],
          [0.6900]],

         [[0.9900],
          [0.1900],
          [0.9900],
          [0.3200],
          [0.9900]],

         [[0.7900],
          [0.9300],
          [0.5500],
          [0.4700],
          [0.0800]]],


        [[[0.7100],
          [0.1900],
          [0.2900],
          [0.3100],
          [0.3800]],

         [[0.7100],
          [0.0700],
          [0.6800],
          [0.3900],
          [0.4900]],

         [[0.5900],
          [0.5000],
          [0.0500],
          [0.8100],
          [0.1700]]]])

In [14]:
motion_embeddings = token.flatten(1,2)
print(motion_embeddings.shape)
print(motion_embeddings)
query = motion_embeddings

torch.Size([2, 15, 1])
tensor([[[0.2900],
         [0.6300],
         [0.4900],
         [0.5200],
         [0.6900],
         [0.9900],
         [0.1900],
         [0.9900],
         [0.3200],
         [0.9900],
         [0.7900],
         [0.9300],
         [0.5500],
         [0.4700],
         [0.0800]],

        [[0.7100],
         [0.1900],
         [0.2900],
         [0.3100],
         [0.3800],
         [0.7100],
         [0.0700],
         [0.6800],
         [0.3900],
         [0.4900],
         [0.5900],
         [0.5000],
         [0.0500],
         [0.8100],
         [0.1700]]])


In [15]:
from src.modeling.modules.lm_utils import get_attention_mask

attn_mask = get_attention_mask(5, query.shape[1])
print(attn_mask.shape)
print(attn_mask)

torch.Size([15, 15])
tensor([[False,  True,  True,  True,  True, False,  True,  True,  True,  True,
         False,  True,  True,  True,  True],
        [False, False,  True,  True,  True, False, False,  True,  True,  True,
         False, False,  True,  True,  True],
        [False, False, False,  True,  True, False, False, False,  True,  True,
         False, False, False,  True,  True],
        [False, False, False, False,  True, False, False, False, False,  True,
         False, False, False, False,  True],
        [False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False],
        [False,  True,  True,  True,  True, False,  True,  True,  True,  True,
         False,  True,  True,  True,  True],
        [False, False,  True,  True,  True, False, False,  True,  True,  True,
         False, False,  True,  True,  True],
        [False, False, False,  True,  True, False, False, False,  True,  True,
         False, False, False,  T

In [33]:
padding_mask = torch.randint(0, 2, [2, 3, 5]).flatten(1, -1)
print(padding_mask.shape)
print(padding_mask)

torch.Size([2, 15])
tensor([[0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0],
        [0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]])


In [21]:
import torch.nn as nn

self_attn = nn.MultiheadAttention(embed_dim = 1, num_heads = 1, batch_first=True)

In [34]:
attn_out_1, _ = self_attn(query, query, query, attn_mask=attn_mask, key_padding_mask = padding_mask.bool())
print(attn_out_1.shape)
print(attn_out_1)

torch.Size([2, 15, 1])
tensor([[[0.1288],
         [0.1348],
         [0.1352],
         [0.1235],
         [0.1198],
         [0.1228],
         [0.1375],
         [0.1329],
         [0.1243],
         [0.1167],
         [0.1245],
         [0.1329],
         [0.1349],
         [0.1237],
         [0.1260]],

        [[0.1727],
         [0.1123],
         [0.0855],
         [0.1010],
         [0.0915],
         [0.1727],
         [0.1131],
         [0.0819],
         [0.1002],
         [0.0904],
         [0.1727],
         [0.1103],
         [0.0877],
         [0.0960],
         [0.0935]]], grad_fn=<TransposeBackward0>)


In [10]:
attn_out_2 = attn_out_1.unflatten(dim=1, sizes=(3, 5)).flatten(0, 1)
print(attn_out_2.shape)
print(attn_out_2)

torch.Size([6, 5, 1])
tensor([[[-0.0537],
         [-0.0602],
         [-0.0839],
         [-0.0966],
         [-0.1006]],

        [[-0.0534],
         [-0.0605],
         [-0.0844],
         [-0.0968],
         [-0.1008]],

        [[-0.0532],
         [-0.0606],
         [-0.0843],
         [-0.0967],
         [-0.1008]],

        [[-0.0551],
         [-0.0532],
         [-0.0714],
         [-0.0936],
         [-0.0828]],

        [[-0.0553],
         [-0.0533],
         [-0.0723],
         [-0.0931],
         [-0.0825]],

        [[-0.0552],
         [-0.0532],
         [-0.0727],
         [-0.0934],
         [-0.0826]]], grad_fn=<UnsafeViewBackward0>)


In [11]:
fused_emb = torch.randint(100, (6, 10, 1)) / 100.
fused_emb

tensor([[[0.9000],
         [0.5100],
         [0.2600],
         [0.2400],
         [0.4700],
         [0.8200],
         [0.0900],
         [0.7400],
         [0.8800],
         [0.7900]],

        [[0.8700],
         [0.1100],
         [0.9900],
         [0.6200],
         [0.3300],
         [0.2100],
         [0.7300],
         [0.4600],
         [0.8400],
         [0.7100]],

        [[0.1400],
         [0.9400],
         [0.3500],
         [0.8800],
         [0.9200],
         [0.7500],
         [0.0600],
         [0.3700],
         [0.7800],
         [0.9900]],

        [[0.5700],
         [0.8500],
         [0.7900],
         [0.5300],
         [0.1200],
         [0.1300],
         [0.8700],
         [0.5700],
         [0.4700],
         [0.6300]],

        [[0.6900],
         [0.8000],
         [0.1600],
         [0.8700],
         [0.8400],
         [0.7300],
         [0.2600],
         [0.6700],
         [0.8600],
         [0.2500]],

        [[0.6400],
         [0.0100],
  

In [12]:
cross_attn = nn.MultiheadAttention(embed_dim = 1, num_heads = 1, batch_first=True)

In [13]:
query, _ = cross_attn(attn_out_2, fused_emb, fused_emb)
print(query.shape)
print(query)

torch.Size([6, 5, 1])
tensor([[[0.3561],
         [0.3561],
         [0.3563],
         [0.3565],
         [0.3565]],

        [[0.3667],
         [0.3667],
         [0.3670],
         [0.3671],
         [0.3671]],

        [[0.3862],
         [0.3863],
         [0.3866],
         [0.3868],
         [0.3868]],

        [[0.3454],
         [0.3453],
         [0.3455],
         [0.3456],
         [0.3456]],

        [[0.3828],
         [0.3828],
         [0.3830],
         [0.3831],
         [0.3831]],

        [[0.3663],
         [0.3663],
         [0.3665],
         [0.3668],
         [0.3667]]], grad_fn=<TransposeBackward0>)


In [14]:
query = query.unflatten(dim=0, sizes=(2, 3)).flatten(1, 2)
print(query.shape)
print(query)

torch.Size([2, 15, 1])
tensor([[[0.3561],
         [0.3561],
         [0.3563],
         [0.3565],
         [0.3565],
         [0.3667],
         [0.3667],
         [0.3670],
         [0.3671],
         [0.3671],
         [0.3862],
         [0.3863],
         [0.3866],
         [0.3868],
         [0.3868]],

        [[0.3454],
         [0.3453],
         [0.3455],
         [0.3456],
         [0.3456],
         [0.3828],
         [0.3828],
         [0.3830],
         [0.3831],
         [0.3831],
         [0.3663],
         [0.3663],
         [0.3665],
         [0.3668],
         [0.3667]]], grad_fn=<UnsafeViewBackward0>)


In [49]:
target_valid = torch.randint(0, 2, [2, 8, 25])
if target_valid.shape[-1] != 55:
    target_valid = torch.cat([target_valid, torch.ones([2, 8, 55 - target_valid.shape[-1]])], dim=-1)
torch.randn([2, 8, 55, 1])[~target_valid.bool()].shape

torch.Size([198, 1])